In [7]:
# Libs Installation
!pip3 install yfinance==0.2.41
!pip3 install crewai==0.28.8
!pip3 install 'crewai[tools]'
!pip3 install langchain==0.1.20
!pip3 install langchain-openai==0.1.7
!pip3 install langchain-community==0.0.38
!pip3 install duckduckgo-search==5.3.0

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.87
    Uninstalling langsmith-0.0.87:
      Successfully uninstalled langsmith-0.0.87
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.20
    Uninstalling langchain-community-0.0.20:
      Successfully uninstalled langchain-community-0.0.20
  Attempting uninstall: langchain
    Foun

In [8]:
# Libs import
import json
import os
from datetime import datetime

import yfinance as yf

from crewai import Agent, Task, Crew, Process

from langchain.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchResults

from IPython.display import Markdown

TypeError: unsupported operand type(s) for |: 'type' and 'NoneType'

In [4]:
# Creation Yahoo Finance Tool
def fetch_stock_price(ticket):
  stock = yf.download(ticket, start="2023-08-08", end="2024-08-08")
  return ticket

yahoo_finance_tool = Tool(
  name="Yahoo Finance Tool",
  description="Fetches stock prices for {ticket} from the last year about a specific company from Yahoo Finance API",
  func=lambda ticket: fetch_stock_price(ticket)
)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Importing OpenAI LLM - GPT
# os.environ['OPENAI_API_KEY'] = ""
llm = ChatOpenAI(model="gpt-3.5-turbo")


In [ ]:
stockPriceAnalyst = Agent(
  role="Senior stock price Analyst",
  goal="Find the {ticket} stock price and analyses trends",
  backstory="""You're highly experienced in analyzing the price of a specific stock
  and make predictions about its future price """,
  verbose=True,
  llm=llm,
  max_iter=5,
  memory=True,
  tools=[yahoo_finance_tool]
)

In [ ]:
getStockPrice = Task(
  description="Analyze the stock {ticket} price history and create a trend analyses of up, down or sideways",
  expected_output=""" Specify the current trend stock price - up, down or sideways.
  eg. stock='APPL', price UP """,
  agent=stockPriceAnalyst
)

In [ ]:
# Importing a search tool
search_tool = DuckDuckGoSearchResults(backend='news', num_results=10)

In [ ]:
newsAnalyst = Agent(
  role="Stock News Analyst",
  goal="""Create a short summary of the market news related to the stock {ticket} company. 
  Specify the current trend - up, down or sideways with the news context. 
  For each request stock asset, specify a number between 0 and 100, where 0 is extreme fear and 100 is extreme greed. """,
  backstory="""You're highly experienced in analyzing the market trends and news and have tracked assest for more then 10 years.

  You're also master level analyts in the tradicional markets and have deep understanding of human psychology.

  You understand news, theirs tittles and information, but you look at those with a health dose of skepticism. 
  You consider also the source of the news articles. 
  """,
  verbose=True,
  llm=llm,
  max_iter=10,
  memory=True,
  tools=[] 
)

In [ ]:
get_news = Task(
    description= f"""Take the stock and always include BTC to it (if not request).
    Use the search tool to search each one individually. 

    The current date is {datetime.now()}.

    Compose the results into a helpfull report""",
    expected_output = """"A summary of the overall market and one sentence summary for each request asset. 
    Include a fear/greed score for each asset based on the news. Use format:
    <STOCK ASSET>
    <SUMMARY BASED ON NEWS>
    <TREND PREDICTION>
    <FEAR/GREED SCORE>
""",
    agent= newsAnalyst
)

In [ ]:
stockAnalystWrite = Agent(
    role = "Senior Stock Analyts Writer",
    goal= """"Analyze the trends price and news and write an insighfull compelling and informative 3 paragraph long newsletter based on the stock report and price trend. """,
    backstory= """You're widely accepted as the best stock analyst in the market. You understand complex concepts and create compelling stories
    and narratives that resonate with wider audiences. 

    You understand macro factors and combine multiple theories - eg. cycle theory and fundamental analyses. 
    You're able to hold multiple opinions when analyzing anything.
    """,
    verbose = True,
    llm=llm,
    max_iter = 5,
    memory=True,
    allow_delegation = True
)

In [ ]:
writeAnalyses = Task(
    description = """Use the stock price trend and the stock news report to create an analyses and write the newsletter about the {ticket} company
    that is brief and highlights the most important points.
    Focus on the stock price trend, news and fear/greed score. What are the near future considerations?
    Include the previous analyses of stock trend and news summary.
""",
    expected_output= """"An eloquent 3 paragraphs newsletter formated as markdown in an easy readable manner. It should contain:

    - 3 bullets executive summary 
    - Introduction - set the overall picture and spike up the interest
    - main part provides the meat of the analysis including the news summary and fead/greed scores
    - summary - key facts and concrete future trend prediction - up, down or sideways.
""",
    agent = stockAnalystWrite,
    context = [getStockPrice, get_news]
)

In [ ]:
crew = Crew(
  agents=[stockPriceAnalyst, newsAnalyst, stockAnalystWrite],
  tasks=[getStockPrice, get_news, writeAnalyses],
  verbose=2,
  process=Process.hierarchical,
  full_output=True,
  share_crew=False,
  manager_llm=llm,
  max_iter=15
)

In [ ]:
results = crew.kickoff(inputs={'ticket': 'AAPL'})

In [ ]:
list(results.key())

In [ ]:
Markdown(results['final_output'])